<a href="https://colab.research.google.com/github/Heity94/TWSM_Lab/blob/main/Project/Notebooks/JE_sentence_trasnformers_TripletsDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Declare whether you are on Colab or local
colab = True

In [ ]:
if colab==True:
  
  #Mount drive
  from google.colab import drive
  drive.mount('/content/drive')


  #install required packages
  #!pip install duckdb -q
  !pip install -U sentence-transformers -q

Mounted at /content/drive
     |████████████████████████████████| 85 kB 4.5 MB/s 
     |████████████████████████████████| 4.4 MB 68.6 MB/s 
     |████████████████████████████████| 1.2 MB 73.4 MB/s 
     |████████████████████████████████| 101 kB 12.0 MB/s 
     |████████████████████████████████| 596 kB 30.6 MB/s 
     |████████████████████████████████| 6.6 MB 81.1 MB/s 


In [ ]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryClassificationEvaluator, TripletEvaluator, LabelAccuracyEvaluator

In [ ]:
#set path to data in Google Drive
data_path = "/content/drive/MyDrive/2022_Analytics Lab Student Projects/Data/All Topics"
data_path_group = data_path[:-10]+"Topic 1/Data_Team1/SynDet/Triplets_Dataset/"

In [ ]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
from datetime import datetime
import os
import gzip
import csv
import pandas as pd

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [40]:
#Load train dataset
train = pd.read_csv(data_path_group+"SynDet_ontology_train_triplets.csv")

In [44]:
train.head()

,Unnamed: 0,entity_id,positive_example,negative_example
0,0,electronic health record,online medical records,postal banking
1,1,archival research,government data sources analysis,action design research
2,2,communication service infrastructure,digital enabled communication computers,model driven development
3,3,survey,quantitative based supervised polling techniques,hierarchical Markov model
4,4,three validities,test convergent and design validity,binary classification


In [ ]:
#Load validation dataset
eval = pd.read_csv(data_path_group+"SynDet_ontology_val_triplets.csv")

In [ ]:
eval.head()

,Unnamed: 0,entity_id,positive_example,negative_example
0,0,autoregressive moving average model,ARMA modelings,survival analysis
1,1,bibliometric analysis,metascience,replication study
2,2,communication service infrastructure,information and communications technology medi...,IT productivity paradox
3,3,individual participant,IS and information senior users,organizational participant
4,5,three validities,diagnostic diagnostic and incremental validities,multitrait multimethod analyis


In [101]:
train_samples = []
for index, row in train.iterrows():
  inp_example = InputExample(texts=[row['entity_id'], row['positive_example'],row['negative_example']])
  train_samples.append(inp_example)

In [99]:
evaluation_samples = []
for index, row in eval.iterrows():
  inp_example = InputExample(texts=[row['entity_id'], row['positive_example'], row['negative_example']])                                   
  evaluation_samples.append(inp_example)

In [100]:
test_samples = []
for index, row in test.iterrows():
  inp_example = InputExample(texts=[row['entity_id'], row['positive_example'], row['negative_example']])                                   
  test_samples.append(inp_example)

In [102]:
train_dataloader = DataLoader(train_samples, drop_last=True, batch_size=32)
train_loss = losses.BatchAllTripletLoss(model=model)

In [109]:
num_epochs = 2
warmup_steps = int(len(train_dataloader) * num_epochs  * 0.1)  # 10% of train data #10% 
#logging.info("Warmup-steps: {}".format(warmup_steps))

In [110]:
warmup_steps

153

In [111]:
evaluator = TripletEvaluator.from_input_examples(evaluation_samples)

In [112]:
model.fit(train_objectives=[(train_dataloader, train_loss)], 
          epochs=num_epochs, 
          warmup_steps=warmup_steps, 
          evaluator=evaluator, 
          evaluation_steps=5000, 
          output_path=data_path)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/766 [00:00<?, ?it/s]

Iteration:   0%|          | 0/766 [00:00<?, ?it/s]

In [98]:
data_path

'/content/drive/MyDrive/2022_Analytics Lab Student Projects/Data/All Topics'

In [113]:
model = SentenceTransformer(data_path)
test_evaluator = TripletEvaluator.from_input_examples(test_samples)
test_evaluator(model, output_path=data_path)

0.949671772428884